# Load data

Make sure you have .env file in ttools or any parent dir with your Alpaca keys.

```bash
ACCOUNT1_LIVE_API_KEY=api_key
ACCOUNT1_LIVE_SECRET_KEY=secret_key
```

###  Cache directories

Daily trade files - DATADIR/tradecache
Agg data cache - DATADIR/aggcache

`DATADIR` - user_data_dir from appdirs library - see `config.py`

In [1]:
import pandas as pd
import numpy as np
from ttools.utils import AggType
from datetime import datetime
from ttools.aggregator_vectorized import generate_time_bars_nb, aggregate_trades
from ttools.loaders import load_data, prepare_trade_cache
from ttools.utils import zoneNY
import vectorbtpro as vbt
from lightweight_charts import PlotDFAccessor, PlotSRAccessor


vbt.settings.set_theme("dark")
vbt.settings['plotting']['layout']['width'] = 1280
vbt.settings.plotting.auto_rangebreaks = True
# Set the option to display with pagination
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_rows', 10)  # Number of rows per page


TTOOLS: Loaded env variables from file /Users/davidbrazda/Documents/Development/python/.env


# Fetching aggregated data

Available aggregation types: 
- time based bars - AggType.OHLCV
- volume based bars - AggType.OHLCV_VOL, resolution = volume threshold
- dollar based bars - AggType.OHLCV_DOL, resolution = dollar threshold
- renko bars - AggType.OHLCV_RENKO resolution = bricksize

In [15]:
#This is how to call LOAD function
symbol = ["BAC","AAPL"]
#datetime in zoneNY 
day_start = datetime(2024, 10, 14, 9, 45, 0)
day_stop = datetime(2024, 10, 16, 15, 1, 0)
day_start = zoneNY.localize(day_start)
day_stop = zoneNY.localize(day_stop)

#requested AGG
resolution = 12 #12s bars
agg_type = AggType.OHLCV #other types AggType.OHLCV_VOL, AggType.OHLCV_DOL, AggType.OHLCV_RENKO
exclude_conditions = ['C','O','4','B','7','V','P','W','U','Z','F','9','M','6'] #None to defaults
minsize = 100 #min trade size to include
main_session_only = True
force_remote = False

data = load_data(symbol = symbol,
                     agg_type = agg_type,
                     resolution = resolution,
                     start_date = day_start,
                     end_date = day_stop,
                     #exclude_conditions = None,
                     minsize = minsize,
                     main_session_only = main_session_only,
                     force_remote = force_remote,
                     return_vbt = True, #returns vbt object
                     verbose = False
                     )

data.ohlcv.data[symbol[0]].lw.plot()


/Users/davidbrazda/Documents/Development/python/ttools/.venv/lib/python3.10/site-packages/vectorbtpro/data/base.py:1905: UserWarning: Symbols have mismatching index. Setting missing data points to NaN.
  data = cls_or_self.align_index(data, missing=missing_index, silence_warnings=silence_warnings)


In [5]:
data.ohlcv.data[symbol[0]]

,open,high,low,close,volume
time,,,,,
2024-10-14 09:45:00-04:00,41.9650,41.970,41.950,41.9500,17895.0
2024-10-14 09:45:12-04:00,41.9589,41.965,41.950,41.9650,6281.0
2024-10-14 09:45:24-04:00,41.9650,42.005,41.965,41.9975,3522.0
2024-10-14 09:45:36-04:00,41.9900,42.005,41.990,42.0000,5960.0
2024-10-14 09:45:48-04:00,42.0050,42.040,42.005,42.0300,9113.0
...,...,...,...,...,...
2024-10-16 15:00:00-04:00,42.9150,42.915,42.910,42.9100,12872.0
2024-10-16 15:00:12-04:00,42.9150,42.920,42.910,42.9200,7574.0
2024-10-16 15:00:24-04:00,42.9200,42.920,42.910,42.9200,1769.0


## Prepare daily trade cache
This is how to prepare trade cache for given symbol and period (if daily trades are not cached they are remotely fetched.)

In [ ]:
symbols = ["BAC", "AAPL"]
#datetime in zoneNY 
day_start = datetime(2024, 10, 1, 9, 45, 0)
day_stop = datetime(2024, 10, 27, 15, 1, 0)
day_start = zoneNY.localize(day_start)
day_stop = zoneNY.localize(day_stop)
force_remote = False

prepare_trade_cache(symbols, day_start, day_stop, force_remote, verbose = True)

### Prepare daily trade cache - cli script

Python script prepares trade cache for specified symbols and date range.

Usually 1 day takes about 35s. It is stored in /tradescache/ directory as daily file keyed by symbol.

To run this script in the background with specific arguments:

```bash
# Running without forcing remote fetch
python3 prepare_cache.py --symbols BAC AAPL --day_start 2024-10-14 --day_stop 2024-10-18 &

# Running with force_remote set to True
python3 prepare_cache.py --symbols BAC AAPL --day_start 2024-10-14 --day_stop 2024-10-18 --force_remote &

```